In [19]:
import argparse
import numpy as np
import networkx as nx
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import pickle
import random

In [31]:
from nltk.cluster import KMeansClusterer
import nltk
from collections import defaultdict
from gensim.models import KeyedVectors

In [70]:
GRAPH_PATH='../data/stats.stackexchange.com/Mixed/Userid_Ngram_Folded_Graph.edges'
EMB_PATH='../data/stats.stackexchange.com/Mixed/node2vec/Userid_Ngram_Folded_Graph.emb'
EMB_PATH_10='../data/stats.stackexchange.com/Mixed/node2vec/Userid_Ngram_Folded_Graph_epoch10.emb'

NGRAMID_DICT_PICKLE_PATH='../data/stats.stackexchange.com/Mixed/ngram_dict_STATS_20k-Posts_11-top_uni&bigrams_nostem'

In [ ]:
# draw the graph DOES NOT WORK

def read_graph(edgelist):
    '''
    Reads the input network in networkx.
    '''
    G = nx.read_edgelist(edgelist, nodetype=int, create_using=nx.DiGraph())
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1
    G = G.to_undirected()
    return G

# load the graph
nx_G = read_graph(GRAPH_PATH)

NUM_DRAW_NODES = 10
DRAW_PATH = '../fig/stats_mixed_ngrams_%d.png' % NUM_DRAW_NODES

ngramid_dict = pickle.load(open(NGRAMID_DICT_PICKLE_PATH, "rb"))
idngram_dict = {v: k for k, v in ngramid_dict.iteritems()}
nx.draw_networkx(nx_G, labels=idngram_dict, nodelist=random.sample(list(nx_G.nodes()), NUM_NODES))
plt.savefig(DRAW_PATH)

In [32]:
# load word vectors
wv = KeyedVectors.load_word2vec_format(EMB_PATH, binary=False)
X = wv[wv.vocab]

In [76]:
wv_e10 = KeyedVectors.load_word2vec_format(EMB_PATH_10, binary=False)
X_e10 = wv_e10[wv_e10.vocab]

In [ ]:
NUM_CLUSTERS=250

In [38]:
# kmeans clustering
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=1)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

In [81]:
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=1,avoid_empty_clusters=True)
assigned_clusters_e10 = kclusterer.cluster(X_e10, assign_clusters=True)

In [63]:
def get_average(nodes):
    clust_X = [wv.get_vector(str(n)) for n in nodes]
    ave = np.average(clust_X, axis=0)
    return ave
ave = get_average(cluster_to_nodes[0])
top10 = wv.similar_by_vector(ave)
print [idngram_dict[int(w[0])] for w in top10]

['points data', 'comparison occur', 'appreciated reproducible', 'survey predicts', 'learning model', 'ucla edu', 'variable measures', 'single regression', 'assign flags', 'juvenile sex']


In [73]:
# manual inspection
def write_clusters(dict_path, words_path, assignments):
    clusters_dict = {}
    for w, clust in zip(wv.vocab, assignments):
        clusters_dict[int(w)] = clust
    
    # save cluster mapping
    with open(dict_path, 'w') as outfile:
        pickle.dump(clusters_dict, outfile)
    
    cluster_to_nodes = defaultdict(list)
    for node in clusters_dict:
        cluster_to_nodes[clusters_dict[node]].append(node)

    with open(words_path, 'w') as outfile:
        for clust in cluster_to_nodes:  
            outfile.write("Cluster %d [%d]\n" % (clust, len(cluster_to_nodes[clust])))
            outfile.write('----------------\n')
            ave = get_average(cluster_to_nodes[clust])
            top10 = wv.similar_by_vector(ave)
            outfile.write('Top words: %s\n' % [idngram_dict[int(w[0])] for w in top10])
            for nid in cluster_to_nodes[clust]:
                outfile.write(idngram_dict[nid]+'\n')
            outfile.write('----------------\n')
            outfile.write('----------------\n')

In [ ]:
CLUSTERS_PATH = '../data/stats.stackexchange.com/Mixed/node2vec_clusters_%d.words' % NUM_CLUSTERS
CLUSTERS_DICT_PATH = '../data/stats.stackexchange.com/Mixed/node2vec_clusters_%d.dict' % NUM_CLUSTERS
write_clusters(CLUSTERS_PATH, CLUSTERS_DICT_PATH, assigned_clusters)

In [82]:
CLUSTERS_PATH_10 = '../data/stats.stackexchange.com/Mixed/node2vec_clusters_%d_epoch10.words' % NUM_CLUSTERS
CLUSTERS_DICT_PATH_10 = '../data/stats.stackexchange.com/Mixed/node2vec_clusters_%d_epoch10.dict' % NUM_CLUSTERS
write_clusters(CLUSTERS_PATH_10, CLUSTERS_DICT_PATH_10, assigned_clusters_e10)

TypeError: write_clusters() takes exactly 2 arguments (3 given)

In [ ]:
# modularity
import get_modularity from get_modularity

G = # to snap graph
mod = get_modularity()